In [1]:
import pandas as pd
import numpy as np
import matplotlib

from readCanLitefileUtils import *
from calibUtils import *

In [2]:
# define the inputs

#file 1 (DACset 0x000)
file_DACset000 = './files/SN04_calib-test/21-05-18_121010_validcalibparam_SN04_dacset000.csv'

#file 2 (DACset 0x3FF)
file_DACset3FF = './files/SN04_calib-test/21-05-18_121010_validcalibparam_SN04_dacset3FF.csv'

#file 3 (DACset 0xCFF)
file_DACsetCFF = './files/SN04_calib-test/21-05-18_121010_validcalibparam_SN04_dacsetCFF.csv'

#file 4 (DACset 0xFFF)
file_DACsetFFF = './files/SN04_calib-test/21-05-18_121010_validcalibparam_SN04_dacsetFFF.csv'


DMM_000 = [108.75, 108.757, 108.742, 108.746, 108.745, 108.745, 
          108.737, 108.743, 108.747, 108.717, 108.727, 108.757]

DMM_3FF = [110.638, 110.643, 110.631, 110.635, 110.636, 110.637,
          110.327, 110.631, 110.640, 110.602, 110.619, 110.651]

DMM_CFF = [114.908, 114.915, 114.895, 114.895, 114.896, 114.892,
          114.892, 114.897, 114.902, 114.872, 114.882, 114.922]


DMM_FFF = [116.329, 116.334, 116.315, 116.319, 116.322, 116.313,
          116.310, 116.317, 116.322, 116.293, 116.302, 116.343]


MM_CFF = [114.908, 114.915, 114.895, 114.895, 114.896, 114.892,
          114.892, 114.897, 114.902, 114.872, 114.882, 114.922]


DMM_FFF = [116.329, 116.334, 116.315, 116.319, 116.322, 116.313,
          116.310, 116.317, 116.322, 116.293, 116.302, 116.343]


values = ['ADC000', 'ADC3FF', 'ADCCFF', 'ADCFFF']

In [6]:
DACset000, ADC000 = GetADCvalues(file_DACset000)

DACset3FF, ADC3FF = GetADCvalues(file_DACset3FF)

DACsetCFF, ADCCFF = GetADCvalues(file_DACsetCFF)

DACsetFFF, ADCFFF = GetADCvalues(file_DACsetFFF)



In [7]:
df = pd.DataFrame(list(zip(ADC000, DMM_000, ADC3FF, DMM_3FF, ADCCFF, DMM_CFF, ADCFFF, DMM_FFF)), 
    columns = ['ADC000', 'DMM000' ,  'ADC3FF', 'DMM3FF', 'ADCCFF', 'DMMCFF', 'ADCFFF', 'DMMFFF'])
df

,ADC000,DMM000,ADC3FF,DMM3FF,ADCCFF,DMMCFF,ADCFFF,DMMFFF
0,e536,108.750,e91f,110.638,f223,114.908,f521,116.329
1,e532,108.757,e92c,110.643,f22d,114.915,f52a,116.334
2,e52b,108.742,e927,110.631,f224,114.895,f523,116.315
3,e52f,108.746,e92a,110.635,f225,114.895,f524,116.319
4,e52c,108.745,e928,110.636,f223,114.896,f524,116.322
5,e52e,108.745,e92b,110.637,f224,114.892,f521,116.313
6,e529,108.737,e925,110.327,f222,114.892,f520,116.310
7,e52c,108.743,e927,110.631,f225,114.897,f523,116.317
8,e51e,108.747,e931,110.640,f235,114.902,f530,116.322
9,e51f,108.717,e918,110.602,f217,114.872,f515,116.293


In [8]:
df_calibparam = pd.read_csv('./calibDB/calib_biasV_SN04_v6_forcheck.csv',delimiter=',') 
df_calibparam.drop('Unnamed: 0', inplace=True, axis=1)


In [9]:
def testcalib_biasV(df, df_calibparam, values):
    df = df.copy()
    a_coeff = 1.8535e-3
    b_coeff = 0
    LSBres = 1.8535e-3


    for val in values:
        #print(val)
        val_calib = val + '_calib'
    
        
        #compute ADC calibrated value in dec
        df[val] = df[val].apply(lambda x: int(x,16) )
        df[val_calib] = ( df[val] + df_calibparam['Offset_Sig BiasV']) * (df_calibparam['Gain_Usig BiasV'] /32768) * a_coeff + b_coeff

       

        #compare with the DMM value
        tmp_DMMlabel = val.replace('ADC', 'DMM')
        val_err = val + 'err'
        df[val_err] = df[tmp_DMMlabel]-df[val_calib]

        
        #compute error 
        val_sig = val + 'sigma'
        df[val_sig] = df[val_err]/LSBres
        #df['ADCmin sigma '] = df['ADCmin err']/LSBres
        #df['ADCmax sigma '] = df['ADCmax err']/LSBres
        
    return df
    

In [10]:
df_aftercalibtest = testcalib_biasV(df, df_calibparam, values)
df_aftercalibtest 

,ADC000,DMM000,ADC3FF,DMM3FF,ADCCFF,DMMCFF,ADCFFF,DMMFFF,ADC000_calib,ADC000err,ADC000sigma,ADC3FF_calib,ADC3FFerr,ADC3FFsigma,ADCCFF_calib,ADCCFFerr,ADCCFFsigma,ADCFFF_calib,ADCFFFerr,ADCFFFsigma
0,58678,108.750,59679,110.638,61987,114.908,62753,116.329,108.045652,0.704348,380.009740,109.888322,0.749678,404.465948,114.136957,0.771043,415.992954,115.547033,0.781967,421.886957
1,58674,108.757,59692,110.643,61997,114.915,62762,116.334,108.032404,0.724596,390.933932,109.906426,0.736574,397.396243,114.149669,0.765331,412.911439,115.557947,0.776053,418.696220
2,58667,108.742,59687,110.631,61988,114.895,62755,116.315,108.017647,0.724353,390.802713,109.895063,0.735937,397.052834,114.130291,0.764709,412.575754,115.542034,0.772966,417.030701
3,58671,108.746,59690,110.635,61989,114.895,62756,116.319,108.042362,0.703638,379.626441,109.918398,0.716602,386.620826,114.150986,0.744014,401.410471,115.563076,0.755924,407.836241
4,58668,108.745,59688,110.636,61987,114.896,62756,116.322,107.996336,0.748664,403.919075,109.874213,0.761787,410.999213,114.106800,0.789200,425.788858,115.522572,0.799428,431.307096
5,58670,108.745,59691,110.637,61988,114.892,62753,116.313,108.017314,0.727686,392.601021,109.896917,0.740083,399.289709,114.125562,0.766438,413.508525,115.533883,0.779117,420.349000
6,58665,108.737,59685,110.327,61986,114.892,62752,116.310,108.019488,0.717512,387.112071,109.896903,0.430097,232.045765,114.132131,0.759869,409.964153,115.542034,0.767966,414.333102
7,58668,108.743,59687,110.631,61989,114.897,62755,116.317,108.014747,0.728253,392.907174,109.890782,0.740218,399.362039,114.128893,0.768107,414.408944,115.539142,0.777858,419.669921
8,58654,108.747,59697,110.640,62005,114.902,62768,116.322,107.866268,0.880732,475.172248,109.791858,0.848142,457.589330,114.052895,0.849105,458.108893,115.461548,0.860452,464.230802
9,58655,108.717,59672,110.602,61975,114.872,62741,116.293,107.921066,0.795934,429.422413,109.791636,0.810364,437.207419,114.027549,0.844451,455.597859,115.436455,0.856545,462.123026


In [11]:
df_test = pd.read_csv('./files/30-04-21_SN04calib_test5_DACmaxFFF.csv' ,delimiter=';', header=None)
df_test = df_test.rename(columns={0: 'ch', 1:'ADCmin', 2: 'DMMmin', 3:'ADCmax', 4:'DMMmax'})
df_test

,ch,ADCmin,DMMmin,ADCmax,DMMmax
0,ch0,E6DD,108.752,F6DB,116.327
1,ch1,E6BB,108.755,F6CC,116.332
2,ch2,E6B5,108.740,F6C7,116.314
3,ch3,E6AE,108.744,F6BE,116.318
4,ch4,E6C0,108.744,F6D4,116.320
5,ch5,E6B8,108.744,F6C8,116.312
6,ch6,E6AF,108.735,F6C2,116.309
7,ch7,E6B7,108.741,F6C7,116.315
8,ch8,E60D,108.745,F712,116.320
9,ch9,E6CF,108.715,F6E5,116.291


In [12]:
df_test = df_test.rename(columns={'ADCmin':'ADC000' , 'DMMmin': 'DMM000', 'ADCmax': 'ADCFFF', 'DMMmax':'DMMFFF'})
df_test

,ch,ADC000,DMM000,ADCFFF,DMMFFF
0,ch0,E6DD,108.752,F6DB,116.327
1,ch1,E6BB,108.755,F6CC,116.332
2,ch2,E6B5,108.740,F6C7,116.314
3,ch3,E6AE,108.744,F6BE,116.318
4,ch4,E6C0,108.744,F6D4,116.320
5,ch5,E6B8,108.744,F6C8,116.312
6,ch6,E6AF,108.735,F6C2,116.309
7,ch7,E6B7,108.741,F6C7,116.315
8,ch8,E60D,108.745,F712,116.320
9,ch9,E6CF,108.715,F6E5,116.291


In [13]:
df_calibparam_test = pd.read_csv('calibDB/calib_biasV_SN04_v5_forcheck.csv',delimiter=',') 
df_calibparam_test.drop('Unnamed: 0', inplace=True, axis=1)
values_test =['ADC000', 'ADCFFF']

df_testcalib_test = testcalib_biasV(df_test, df_calibparam_test, values_test)
df_testcalib_test

,ch,ADC000,DMM000,ADCFFF,DMMFFF,ADC000_calib,ADC000err,ADC000sigma,ADCFFF_calib,ADCFFFerr,ADCFFFsigma
0,ch0,59101,108.752,63195,116.327,108.751812,0.000188,0.101231,116.326842,0.000158,0.085444
1,ch1,59067,108.755,63180,116.332,108.753517,0.001483,0.799869,116.330433,0.001567,0.845360
2,ch2,59061,108.740,63175,116.314,108.741964,-0.001964,-1.059408,116.316067,-0.002067,-1.115386
3,ch3,59054,108.744,63166,116.318,108.742347,0.001653,0.891945,116.316257,0.001743,0.940215
4,ch4,59072,108.744,63188,116.320,108.742845,0.001155,0.623207,116.318768,0.001232,0.664579
5,ch5,59064,108.744,63176,116.312,108.745364,-0.001364,-0.735740,116.313459,-0.001459,-0.787382
6,ch6,59055,108.735,63170,116.309,108.735482,-0.000482,-0.260303,116.309565,-0.000565,-0.304929
7,ch7,59063,108.741,63175,116.315,108.740505,0.000495,0.267130,116.314415,0.000585,0.315400
8,ch8,58893,108.745,63250,116.320,108.743531,0.001469,0.792610,116.318442,0.001558,0.840361
9,ch9,59087,108.715,63205,116.291,108.716634,-0.001634,-0.881785,116.292745,-0.001745,-0.941426
